# 使用工具的扩展思考

## 目录
- [设置](#设置)
- [基本示例](#基本示例)
- [多次工具调用](#使用思考进行多次工具调用)
- [保留思考块](#保留思考块)

本笔记本演示了如何将 Claude 3.7 Sonnet 的扩展思考功能与工具一起使用。扩展思考功能使您能够在 Claude 提供最终答案之前看到其逐步的思考过程，透明地展示它如何决定使用哪些工具以及如何解释工具结果。

当使用扩展思考与工具使用时，模型将在发出工具请求之前显示其思考，但不会在接收工具结果后重复思考过程。Claude 不会输出另一个思考块，直到下一个非 `tool_result` 的 `user` 轮次之后。有关扩展思考的更多信息，请参阅我们的[文档](https://docs.claude.com/en/docs/build-with-claude/extended-thinking)。

## 设置

首先，让我们安装必要的包并设置环境。

In [ ]:
%pip install anthropic

In [ ]:
import anthropic
import os
import json

# 模型和令牌预算的全局变量
MODEL_NAME = "claude-sonnet-4-5"
MAX_TOKENS = 4000
THINKING_BUDGET_TOKENS = 2000

# 将API密钥设置为环境变量或直接设置
# os.environ["ANTHROPIC_API_KEY"] = "your_api_key_here"

# 初始化客户端
client = anthropic.Anthropic()

# 辅助函数
def print_thinking_response(response):
    """美观地打印带有思考块的消息响应。"""
    print("\n==== 完整响应 ====")
    for block in response.content:
        if block.type == "thinking":
            print("\n🧠 思考块:")
            # 为可读性显示截断的思考内容 
            print(block.thinking[:500] + "..." if len(block.thinking) > 500 else block.thinking)
            print(f"\n[签名可用: {bool(getattr(block, 'signature', None))}]")
            if hasattr(block, 'signature') and block.signature:
                print(f"[签名（前50个字符）: {block.signature[:50]}...]")
        elif block.type == "redacted_thinking":
            print("\n🔒 编辑过的思考块:")
            print(f"[数据长度: {len(block.data) if hasattr(block, 'data') else 'N/A'}]")
        elif block.type == "text":
            print("\n✓ 最终答案:")
            print(block.text)
    
    print("\n==== 响应结束 ====")

def count_tokens(messages, tools=None):
    """计算带有可选工具的给定消息列表的令牌数。"""
    if tools:
        response = client.messages.count_tokens(
            model=MODEL_NAME,
            messages=messages,
            tools=tools
        )
    else:
        response = client.messages.count_tokens(
            model=MODEL_NAME,
            messages=messages
        )
    return response.input_tokens

## 使用思考进行单次工具调用

此示例演示如何结合思考并发出单次工具调用，使用模拟的天气工具。

In [ ]:
def tool_use_with_thinking():
    # 定义天气工具
    tools = [
        {
            "name": "weather",
            "description": "获取某个位置的当前天气信息。",
            "input_schema": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "要获取天气信息的位置。"
                    }
                },
                "required": ["location"]
            }
        }
    ]
    
    def weather(location):
        # 模拟天气数据
        weather_data = {
            "New York": {"temperature": 72, "condition": "Sunny"},
            "London": {"temperature": 62, "condition": "Cloudy"},
            "Tokyo": {"temperature": 80, "condition": "Partly cloudy"},
            "Paris": {"temperature": 65, "condition": "Rainy"},
            "Sydney": {"temperature": 85, "condition": "Clear"},
            "Berlin": {"temperature": 60, "condition": "Foggy"},
        }
        
        return weather_data.get(location, {"error": f"No weather data available for {location}"})
    
    # 带有工具使用和思考的初始请求
    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=MAX_TOKENS,
        thinking={
            "type": "enabled",
            "budget_tokens": THINKING_BUDGET_TOKENS
        },
        tools=tools,
        messages=[{
            "role": "user",
            "content": "What's the weather like in Paris today?"
        }]
    )
    
    # 初始响应的详细诊断输出
    print("\n=== 初始响应 ===")
    print(f"响应ID: {response.id}")
    print(f"停止原因: {response.stop_reason}")
    print(f"模型: {response.model}")
    print(f"内容块: {len(response.content)} 个块")
    
    for i, block in enumerate(response.content):
        print(f"\n块 {i+1}: 类型 = {block.type}")
        if block.type == "thinking":
            print(f"思考内容: {block.thinking[:150]}...")
            print(f"签名可用: {bool(getattr(block, 'signature', None))}")
        elif block.type == "text":
            print(f"文本内容: {block.text}")
        elif block.type == "tool_use":
            print(f"工具: {block.name}")
            print(f"工具输入: {block.input}")
            print(f"工具ID: {block.id}")
    print("=== 初始响应结束 ===\n")
    
    # 提取思考块以包含在对话历史中
    assistant_blocks = []
    for block in response.content:
        if block.type in ["thinking", "redacted_thinking", "tool_use"]:
            assistant_blocks.append(block)
            
    # 处理工具使用（如果需要）
    full_conversation = [{
        "role": "user",
        "content": "What's the weather like in Paris today?"
    }]
    
    if response.stop_reason == "tool_use":
        # 添加带有思考块和工具使用的完整助手响应
        full_conversation.append({
            "role": "assistant",
            "content": assistant_blocks
        })
        
        # 查找tool_use块
        tool_use_block = next((block for block in response.content if block.type == "tool_use"), None)
        if tool_use_block:
            # 执行工具
            print(f"\n=== 执行工具 ===")
            print(f"工具名称: {tool_use_block.name}")
            print(f"要检查的位置: {tool_use_block.input['location']}")
            tool_result = weather(tool_use_block.input["location"])
            print(f"结果: {tool_result}")
            print("=== 工具执行完成 ===\n")
            
            # 将工具结果添加到对话中
            full_conversation.append({
                "role": "user",
                "content": [{
                    "type": "tool_result",
                    "tool_use_id": tool_use_block.id,
                    "content": json.dumps(tool_result)
                }]
            })
            
            # 使用相同的思考配置继续对话
            print("\n=== 发送带有工具结果的跟进请求 ===")
            response = client.messages.create(
                model=MODEL_NAME,
                max_tokens=MAX_TOKENS,
                thinking={
                    "type": "enabled",
                    "budget_tokens": THINKING_BUDGET_TOKENS
                },
                tools=tools,
                messages=full_conversation
            )
            print(f"收到跟进响应。停止原因: {response.stop_reason}")
    
    print_thinking_response(response)

# 运行示例
tool_use_with_thinking()

## 使用思考进行多次工具调用

此示例演示如何处理多次工具调用（如模拟的新闻和天气服务），同时观察思考过程。

In [ ]:
def multiple_tool_calls_with_thinking():
    # 定义工具
    tools = [
        {
            "name": "weather",
            "description": "获取某个位置的当前天气信息。",
            "input_schema": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "要获取天气信息的位置。"
                    }
                },
                "required": ["location"]
            }
        },
        {
            "name": "news",
            "description": "获取某个主题的最新新闻头条。",
            "input_schema": {
                "type": "object",
                "properties": {
                    "topic": {
                        "type": "string",
                        "description": "要获取新闻的主题。"
                    }
                },
                "required": ["topic"]
            }
        }
    ]
    
    def weather(location):
        # 模拟天气数据
        weather_data = {
            "New York": {"temperature": 72, "condition": "Sunny"},
            "London": {"temperature": 62, "condition": "Cloudy"},
            "Tokyo": {"temperature": 80, "condition": "Partly cloudy"},
            "Paris": {"temperature": 65, "condition": "Rainy"},
            "Sydney": {"temperature": 85, "condition": "Clear"},
            "Berlin": {"temperature": 60, "condition": "Foggy"},
        }
        
        return weather_data.get(location, {"error": f"No weather data available for {location}"})
    
    def news(topic):
        # 模拟新闻数据
        news_data = {
            "technology": [
                "New AI breakthrough announced by research lab",
                "Tech company releases latest smartphone model",
                "Quantum computing reaches milestone achievement"
            ],
            "sports": [
                "Local team wins championship game",
                "Star player signs record-breaking contract",
                "Olympic committee announces host city for 2036"
            ],
            "weather": [
                "Storm system developing in the Atlantic",
                "Record temperatures recorded across Europe",
                "Climate scientists release new research findings"
            ]
        }
        
        return {"headlines": news_data.get(topic.lower(), ["No news available for this topic"])}
    
    # 初始请求
    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=MAX_TOKENS,
        thinking={
                "type": "enabled",
                "budget_tokens": THINKING_BUDGET_TOKENS
        },
        tools=tools,
        messages=[{
            "role": "user",
            "content": "What's the weather in London, and can you also tell me the latest news about technology?"
        }]
    )
    
    # 打印初始响应的详细信息
    print("\n=== 初始响应 ===")
    print(f"响应ID: {response.id}")
    print(f"停止原因: {response.stop_reason}")
    print(f"模型: {response.model}")
    print(f"内容块: {len(response.content)} 个块")
    
    # 打印每个内容块
    for i, block in enumerate(response.content):
        print(f"\n块 {i+1}: 类型 = {block.type}")
        if block.type == "thinking":
            print(f"思考内容: {block.thinking[:150]}...")
            print(f"签名可用: {bool(getattr(block, 'signature', None))}")
        elif block.type == "text":
            print(f"文本内容: {block.text}")
        elif block.type == "tool_use":
            print(f"工具: {block.name}")
            print(f"工具输入: {block.input}")
            print(f"工具ID: {block.id}")
    print("=== 初始响应结束 ===\n")
    
    # 处理可能多次工具调用
    full_conversation = [{
        "role": "user",
        "content": "What's the weather in London, and can you also tell me the latest news about technology?"
    }]
    
    # 跟踪多轮工具使用的迭代计数
    iteration = 0
    
    while response.stop_reason == "tool_use":
        iteration += 1
        print(f"\n=== 工具使用迭代 {iteration} ===")
        
        # 提取思考块和工具使用以包含在对话历史中
        assistant_blocks = []
        for block in response.content:
            if block.type in ["thinking", "redacted_thinking", "tool_use"]:
                assistant_blocks.append(block)
        
        # 添加带有思考块和工具使用的助手响应
        full_conversation.append({
            "role": "assistant",
            "content": assistant_blocks
        })
        
        # 查找tool_use块
        tool_use_block = next((block for block in response.content if block.type == "tool_use"), None)
        if tool_use_block:
            print(f"\n=== 执行工具 ===")
            print(f"工具名称: {tool_use_block.name}")
            
            # 执行适当的工具
            if tool_use_block.name == "weather":
                print(f"要检查的位置: {tool_use_block.input['location']}")
                tool_result = weather(tool_use_block.input["location"])
            elif tool_use_block.name == "news":
                print(f"要检查的主题: {tool_use_block.input['topic']}")
                tool_result = news(tool_use_block.input["topic"])
            else:
                tool_result = {"error": "Unknown tool"}
                
            print(f"结果: {tool_result}")
            print("=== 工具执行完成 ===\n")
            
            # 将工具结果添加到对话中
            full_conversation.append({
                "role": "user",
                "content": [{
                    "type": "tool_result",
                    "tool_use_id": tool_use_block.id,
                    "content": json.dumps(tool_result)
                }]
            })
            
            # 继续对话
            print("\n=== 发送带有工具结果的跟进请求 ===")
            response = client.messages.create(
                model=MODEL_NAME,
                max_tokens=MAX_TOKENS,
                thinking={
                        "type": "enabled",
                        "budget_tokens": THINKING_BUDGET_TOKENS
                },
                tools=tools,
                messages=full_conversation
            )
            
            # 打印跟进响应详情
            print(f"\n=== 跟进响应（迭代 {iteration}）===")
            print(f"响应ID: {response.id}")
            print(f"停止原因: {response.stop_reason}")
            print(f"内容块: {len(response.content)} 个块")
            
            for i, block in enumerate(response.content):
                print(f"\n块 {i+1}: 类型 = {block.type}")
                if block.type == "thinking":
                    print(f"思考内容预览: {block.thinking[:100]}...")
                elif block.type == "text":
                    print(f"文本内容预览: {block.text[:100]}...")
                elif block.type == "tool_use":
                    print(f"工具: {block.name}")
                    print(f"工具输入预览: {str(block.input)[:100]}")
            print(f"=== 跟进响应结束（迭代 {iteration}）===\n")
            
            if response.stop_reason != "tool_use":
                print("\n=== 最终响应 ===")
                print_thinking_response(response)
                print("=== 最终响应结束 ===")
        else:
            print("响应中未找到tool_use块。")
            break

# 运行示例
multiple_tool_calls_with_thinking()

## 保留思考块

在使用扩展思考和工具时，请确保：

1. **保留思考块签名**：每个思考块都包含一个加密签名，用于验证对话上下文。传递思考块给Claude时必须包含这些签名。

2. **避免修改之前的上下文**：如果在提交带有工具结果的新请求时修改了任何之前的内容（包括思考块），API将拒绝请求。

3. **同时处理思考和编辑过的思考块**：两种类型的块都必须保留在对话历史中，即使编辑块的内容不是人类可读的。

有关不使用工具的扩展思考的更多详细信息，请参阅主要的"扩展思考"笔记本。

In [ ]:
def thinking_block_preservation_example():
    # 定义一个简单的天气工具
    tools = [
        {
            "name": "weather",
            "description": "获取某个位置的当前天气信息。",
            "input_schema": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "要获取天气信息的位置。"
                    }
                },
                "required": ["location"]
            }
        }
    ]
    
    def weather(location):
        # 模拟天气数据
        weather_data = {
            "New York": {"temperature": 72, "condition": "Sunny"},
            "London": {"temperature": 62, "condition": "Cloudy"},
            "Tokyo": {"temperature": 80, "condition": "Partly cloudy"},
            "Paris": {"temperature": 65, "condition": "Rainy"},
            "Sydney": {"temperature": 85, "condition": "Clear"},
            "Berlin": {"temperature": 60, "condition": "Foggy"},
        }
        
        return weather_data.get(location, {"error": f"No weather data available for {location}"})
    
    # 带有工具使用和思考的初始请求
    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=MAX_TOKENS,
        thinking={
            "type": "enabled",
            "budget_tokens": THINKING_BUDGET_TOKENS
        },
        tools=tools,
        messages=[{
            "role": "user",
            "content": "What's the weather like in Berlin right now?"
        }]
    )
    
    # 从响应中提取块
    thinking_blocks = [b for b in response.content if b.type == "thinking"]
    tool_use_blocks = [b for b in response.content if b.type == "tool_use"]
    
    print("\n=== 初始响应 ===")
    print(f"响应包含:")
    print(f"- {len(thinking_blocks)} 个思考块")
    print(f"- {len(tool_use_blocks)} 个工具使用块")
    
    # 检查是否触发了工具使用
    if tool_use_blocks:
        tool_block = tool_use_blocks[0]
        print(f"\n调用的工具: {tool_block.name}")
        print(f"要检查的位置: {tool_block.input['location']}")
        
        # 执行工具
        tool_result = weather(tool_block.input["location"])
        print(f"工具结果: {tool_result}")
        
        # 首先，让我们尝试不包含思考块
        print("\n=== 测试1：不包含思考块 ===")
        try:
            # 注意我们只包含tool_use块，不包含思考块
            partial_blocks = tool_use_blocks
            
            incomplete_response = client.messages.create(
                model=MODEL_NAME,
                max_tokens=MAX_TOKENS,
                thinking={
                        "type": "enabled",
                        "budget_tokens": THINKING_BUDGET_TOKENS
                },
                tools=tools,
                messages=[
                    {"role": "user", "content": "What's the weather like in Berlin right now?"},
                    {"role": "assistant", "content": partial_blocks},
                    {"role": "user", "content": [{
                        "type": "tool_result",
                        "tool_use_id": tool_block.id,
                        "content": json.dumps(tool_result)
                    }]}
                ]
            )
            print("成功：未包含思考块的响应（不符合预期）")
        except Exception as e:
            print(f"错误: {e}")
            print("这证明思考块必须被保留")
        
        # 现在尝试包含思考块（正确方法）
        print("\n=== 测试2：包含思考块（正确方法）===")
        try:
            # 包含响应中的所有块
            complete_blocks = thinking_blocks + tool_use_blocks
            
            complete_response = client.messages.create(
                model=MODEL_NAME,
                max_tokens=MAX_TOKENS,
                thinking={
                    "type": "enabled",
                    "budget_tokens": THINKING_BUDGET_TOKENS
                },
                tools=tools,
                messages=[
                    {"role": "user", "content": "What's the weather like in Berlin right now?"},
                    {"role": "assistant", "content": complete_blocks},
                    {"role": "user", "content": [{
                        "type": "tool_result",
                        "tool_use_id": tool_block.id,
                        "content": json.dumps(tool_result)
                    }]}
                ]
            )
            print("成功：包含思考块的响应")
            
            # 检查第二个响应是否包含思考块
            second_thinking = [b for b in complete_response.content if b.type == "thinking"]
            second_text = [b for b in complete_response.content if b.type == "text"]
            
            print(f"\n第二个响应包含:")
            print(f"- {len(second_thinking)} 个思考块")
            print(f"- {len(second_text)} 个文本块")
            
            if second_text:
                print(f"\n最终答案: {second_text[0].text}")
            
            print("\n注意：工具使用后的第二个响应不包含思考块。")
            print("这是预期行为 - 在工具使用前显示思考，但接收工具结果后不显示。")
            
        except Exception as e:
            print(f"错误: {e}")
    
# 取消注释以运行示例
thinking_block_preservation_example()

## 结论

本笔记本展示了如何将Claude的扩展思考功能与工具使用结合。关键优势包括：

1. 在使用工具时对Claude思考过程的透明度
2. 了解Claude如何决定何时使用工具与内部知识
3. 更好地理解涉及多次工具调用的多步骤任务
4. 洞察Claude如何解释工具结果并将其整合到响应中

在使用扩展思考与工具时，请记住：
- 为复杂任务设置适当的思考预算（最少1,024个令牌）
- 传递工具结果时始终保留思考块及其签名
- 在对话历史中包含正常和编辑过的思考块
- 确保系统提示、工具和思考配置在调用之间匹配
- 期望工具结果轮次不会包含额外的思考块
- 工具使用和思考结合可以增加令牌使用量和响应时间